In [1]:
import pandas as pd
import tarfile
import json
import os

from lib import *
from train import *
from demo import *

In [3]:
def read_json_from_tar(tar_file):
    json_data = []
    with tarfile.open(tar_file, 'r:gz') as tar:
        for member in tar.getmembers():
            if member.isfile() and member.name.endswith('.json'):
                file = tar.extractfile(member)
                data = json.load(file)
                json_data.append(data)
    return json_data

In [5]:
data_list = []
directory = 'data/protondb-data/reports'
for file in os.listdir(directory):
    if file.endswith('.tar.gz'):
        file_path = os.path.join(directory, file)
        json_data = read_json_from_tar(file_path)
        data_list.extend(json_data)

In [ ]:
df = pd.DataFrame(data_list)